<h1>Machine Learning - Assignment 2<h1>

In [1157]:
import pandas as pd
from collections import defaultdict, Counter

<h2>Loading the Review Training and Test Data set provided</h2>

In [1158]:
k=1

#k = int(input("Please input value for K: "))
NumTrainClassLabels1 = 0
NumTrainClassLabels0 = 0
    
TrainingWordsSet = []
TrainingClassesSet = []

with open("reviewstrain.txt") as fHandler:
    for line in fHandler:
        rowTokens = line.rstrip("\n\t").split(" ")
        
        TrainingClassesSet.append(rowTokens[0])
        
        if rowTokens[0] == '1':
            NumTrainClassLabels1 += 1
        else:
            NumTrainClassLabels0 += 1 
        
        del rowTokens[0]
        
        TrainingWordsSet.append(list(set(rowTokens)))

<h2> Calculating the total label count of 0's and 1's from the Training data set</h2>

In [1159]:
#print(TrainingClassesSet[5])
#print(TrainingWordsSet[5])

print("Total training samples labelled as 0 : ", NumTrainClassLabels0)
print("Total training samples labelled as 1 : ", NumTrainClassLabels1)


Total training samples labelled as 0 :  689
Total training samples labelled as 1 :  811


In [1160]:
TestWordsSet = []
TestClassesSet = []


with open("reviewstest.txt") as fHandler:
    for line in fHandler:
        rowTokens = line.rstrip("\n\t").split(" ")
        
        TestClassesSet.append(rowTokens[0])
        
        del rowTokens[0]
        
        TestWordsSet.append(list(set(rowTokens)))

In [1161]:
#print(TestWordsSet[5])
#print(TestClassesSet[5])

<h2> Implementation of the distance function provided in the assignment to calculate the distance of a Test data sample from all the samples in the training set </h2>

In [1162]:
#The below function will calculate the distance for the given test data with each of the sample from the training set
#This function will calculate and return an dictiornay with the distance as the index and all the classess that are at that
#distance as the value of the dictionary

def checkForDistance(ExpectedClass, TestData, TrainingClassSet, TrainingDataSet):
    distanceCalcList = defaultdict(list)
    
    index = 0
    
    for data in TrainingDataSet:
        #print (data, TestData)
        matchCount=0
        
        for word in TestData:
            if word in data:
                matchCount+=1
        
        #print(matchCount)
        
        if (matchCount != 0): 
            distanceCalcList[1/matchCount].append(int(TrainingClassSet[index]))
            
        index+=1

    return distanceCalcList

<h2>Function to be used to performing k-NN search for the entire test data setbased on the value of K passed</h2>

In [1163]:
def performNearestNeighbourSearchForTest(k):
    df = pd.DataFrame(columns=['Expected Class', 'Predicted Class'])
    correct = 0
    error = 0
    index1 = 0 #index to be used as reference for Expected class
    FP = 0 #False positive 
    FN = 0 #False Negative 
    TP = 0 #True positive
    TN = 0 #True Negative
    
    for testSample in TestWordsSet:
        ClassLabel = 0 #Label assigned to the current class

        #print(testSample)
        DistanceCalc = checkForDistance(TestClassesSet[index1], testSample, TrainingClassesSet, TrainingWordsSet)

        sortedKeys = sorted(DistanceCalc) 

        #print(sortedKeys)
        #print(DistanceCalc)

        #Instead of doing a direct comparision for all the labels here to check the count of the majority label
        #we do another mathematical computation to make it more efficient
        # i.e. we knew that if there are 4 labels out of which we have to select the majority and the lables are
        #  1 1 1 0 , then we know that the label based on this will be 1 and if the labels as 1 0 0 0 then the label
        #  based on this will be zero. Thus, what we do is we calculate the sum of all the labels and divide it by the total number
        # Labels i.e we add up 1110 which gives us 3 and divide it by the total number of lables i.e 4 
        # thus we get the answer as 3/4 i.e 0.75 based on this we have derived the formula to efficiently calculate it
        # and determine that if the mean of all the labels is greater than or equal to 0.5 we assign the label zero,
        # otherwise we assign them the label zero
            
        
        #Handling in case if k==1
        if (k == 1):
            #print(DistanceCalc)
            if (len(DistanceCalc[sortedKeys[0]])) > 1:
                sumVal = sum(DistanceCalc[sortedKeys[0]])
                Mean= sumVal/(len(DistanceCalc[sortedKeys[0]]))

                if Mean >= 0.5:
                    ClassLabel = 1
                else:
                    ClassLabel = 0
            else:
                ClassLabel = DistanceCalc[sortedKeys[0]][0]
                
        else:
            
            #Handle condition for value of K more than 2
            NN = k
            TotalNN = 0
            sumVal = 0
            
            for key in sortedKeys:
                
                if (len(DistanceCalc[key])) > 1:
                    sumVal += sum(DistanceCalc[key])
                    TotalNN += len(DistanceCalc[key])
                else:
                    sumVal += DistanceCalc[key][0]
                    TotalNN += 1
                    
                NN = NN-len(DistanceCalc[key])
                
                if NN <= 0:
                    #print(NN)
                    break
            
            Mean = sumVal/TotalNN
            
            if Mean >= 0.5:
                ClassLabel = 1
            else:
                ClassLabel = 0
                
            #print(Mean, " ", sumVal, " ", TotalNN, " ", ClassLabel)
                    
        LabelList = [int(TestClassesSet[index1]), ClassLabel]
        
        #print(LabelList, index1)
        tempVal = int(TestClassesSet[index1])
        if (tempVal == ClassLabel):
            correct+=1
        else:
            error+=1
        
        if (tempVal == 1):
            if (ClassLabel == 1):
                TP += 1
            else:
                FN += 1
        else:
            if (ClassLabel == 0):
                TN += 1
            else:
                FP+=1
                
        
        df.loc[len(df)] = LabelList  
        
        #print("ClassLabel ", ClassLabel, TestClassesSet[index])
        
        index1 += 1 
        #print("index 1 ", index1)
    
    print(correct, " ", error)
    
    return df, TP, FN, TN, FP
    

<h2>Performing Nearest neighbour search for k=1</h2>

In [1164]:
k=1 # set the value for nearest neighbour for k=1
df, TP, FN, TN, FP = performNearestNeighbourSearchForTest(k)

302   198


In [1165]:
print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

#Calculate the True Positive rate for k=1
print("True positive rate for k=1 is ", TP/(TP+FN))

#Calculate the False positive rate for k=1
print("False positive rate for k=1 is ", FP/(FP+TN))

#Calculate the accuracy for k=1
print("Accuracy for k=1 is ", ((TP+TN)/(TP+TN+FP+FN)) * 100, "%")

df.head(20)

True  Positive :  209
True  Negative :  93
False Negative :  64
False Positive :  134
True positive rate for k=1 is  0.7655677655677655
False positive rate for k=1 is  0.5903083700440529
Accuracy for k=1 is  60.4 %


,Expected Class,Predicted Class
0,1,1
1,1,0
2,1,1
3,0,1
4,0,0
5,0,0
6,1,1
7,1,1
8,0,0
9,1,1


<h2>Performing Nearest neighbour algorithm for k=5</h2>

In [1166]:
k=5 # set the value for nearest neighbour for k=5
df, TP, FN, TN, FP = performNearestNeighbourSearchForTest(k)

print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

#Calculate the True Positive rate for k=5
print("True positive rate for k=5 is ", TP/(TP+FN))

#Calculate the False positive rate for k=5
print("False positive rate for k=5 is ", FP/(FP+TN))

#Calculate the accuracy for k=5
print("Accuracy for k=5 is ", ((TP+TN)/(TP+TN+FP+FN)) * 100, "%")

df.head(20)

303   197
True  Positive :  212
True  Negative :  91
False Negative :  61
False Positive :  136
True positive rate for k=5 is  0.7765567765567766
False positive rate for k=5 is  0.5991189427312775
Accuracy for k=5 is  60.6 %


,Expected Class,Predicted Class
0,1,1
1,1,0
2,1,1
3,0,1
4,0,0
5,0,0
6,1,1
7,1,1
8,0,0
9,1,1


<h2>Perform Zero-R Algorithm analysis for Predictions</h2>

In [1167]:
def performZeroRAlogPredictions(TestClassesSet):
    ClassLabel = 0
    TP = 0
    TN = 0
    FP = 0
    FN = 0
    Correct = 0
    Error = 0
    
    
    if NumTrainClassLabels1 >= NumTrainClassLabels0:
        ClassLabel = 1
    else:
        ClassLabel = 0
        
    print("Class Label ", ClassLabel)
    
    for val in TestClassesSet:
        Label = int(val)
        
        if ClassLabel==Label:
            Correct+=1
        else:
            Error+=1
        
        if (Label == 1):
            if (ClassLabel == 1):
                TP += 1
            else:
                FN += 1
        else:
            if (ClassLabel == 0):
                TN += 1
            else:
                FP+=1

    return Correct,Error,TP,TN,FP,FN

In [1168]:
Correct,Error,TP,TN,FP,FN = performZeroRAlogPredictions(TestClassesSet)

print("Correct predictions :", Correct)
print("Error predictions :", Error)
print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

Class Label  1
Correct predictions : 273
Error predictions : 227
True  Positive :  273
True  Negative :  0
False Negative :  0
False Positive :  227


<h2> Performing Cross-Validation</h2>

In [1169]:
def performNearestNeighbourSearchForTestCV(TrainingData, TrainingClass, TestData, TestClassSet, results):
    correct = 0
    error = 0
    index1 = 0 #index to be used as reference for Expected class
    
    for testSample in TestData:
        ClassLabel = 0 #Label assigned to the current class

        DistanceCalc = checkForDistance(TestClassSet[index1], testSample, TrainingClass, TrainingData)

        sortedKeys = sorted(DistanceCalc)
        
        #Handling in case if k==1
        for k in results:
            #Handle condition for value of K more than 2
            NN = k
            TotalNN = 0
            sumVal = 0

            for key in sortedKeys:

                if (len(DistanceCalc[key])) > 1:
                    sumVal += sum(DistanceCalc[key])
                    TotalNN += len(DistanceCalc[key])
                else:
                    sumVal += DistanceCalc[key][0]
                    TotalNN += 1

                NN = NN-len(DistanceCalc[key])

                if NN <= 0:
                    #print(NN)
                    break

            Mean = sumVal/TotalNN

            if Mean >= 0.5:
                ClassLabel = 1
            else:
                ClassLabel = 0
                
            if (int(TestClassSet[index1]) == ClassLabel):
                results[k]["correct"]+=1
                correct+=1
            else:
                results[k]["error"]+=1
                error+=1
        
        index1 += 1

In [1170]:
def performTestingForCrossValidation(TrainingData, TrainingClass, TestData, TestClassSet, results):
    performNearestNeighbourSearchForTestCV(TrainingData, TrainingClass, TestData, TestClassSet, results)
    

In [1171]:
#Function to peform 5 fold cross validation
def performCrossValidationForTraining(TrainingWordsData, TrainingClassesData):
    results = {3:{"correct" : 0, "error" : 0}, 7:{"correct" : 0, "error" : 0}, 99:{"correct" : 0, "error" : 0}}
    
    for indexVal in range(0, 1500, 300):
        RangeVal = indexVal + 300
        
        TempTestSet = TrainingWordsData[indexVal: RangeVal]  
        TempTestClassesSet = TrainingClassesData[indexVal : RangeVal]
        
        TempTrainingSet = list(TrainingWordsData)
        TempClassesSet = list(TrainingClassesData)
            
        del TempTrainingSet[indexVal:RangeVal]
        del TempClassesSet[indexVal:RangeVal]
    
        
        performTestingForCrossValidation(TempTrainingSet, TempClassesSet, TempTestSet, TempTestClassesSet, results)
    
    return results

In [1172]:
results = performCrossValidationForTraining(TrainingWordsSet, TrainingClassesSet)

print("Results obtained for different values of K after cross-validation", results)

Results obtained for different values of K after cross-validation {3: {'correct': 991, 'error': 509}, 7: {'correct': 988, 'error': 512}, 99: {'correct': 919, 'error': 581}}


In [1173]:
#Calculating the accuracy for different values of  k i.e 3,7,99 using cross validation
#Accuracy for K=3
print("Accuracy for k =3 ", (results[3]['correct']/1500)*100, "%")

#Accuracy for K=7
print("Accuracy for k =7 ", (results[7]['correct']/1500)*100, "%")

#Accuracy for K=99
print("Accuracy for k = 99 ", (results[99]['correct']/1500)*100, "%")

#We fetch the value of k for which we have obtainend the maximum accuracy
currentMaxVal=0
currentValofK=0

for k in results:
    if results[k]["correct"] > currentMaxVal:
        currentMaxVal = results[k]["correct"]
        currentValofK = k

Accuracy for k =3  66.06666666666666 %
Accuracy for k =7  65.86666666666666 %
Accuracy for k = 99  61.266666666666666 %


In [1174]:
print (currentMaxVal, currentValofK)

991 3


In [1175]:
df, TP, FN, TN, FP = performNearestNeighbourSearchForTest(currentValofK)
print("Analysis for the Test data set using the value of K obtained after cross validation of training dataset")
print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

#calculate the accuracy for k=3 for the entire training set
print("Accuracy for current value of K with maximum accuracy in cross validation is ", ((TP+TN)/500)*100, "%")

295   205
Analysis for the Test data set using the value of K obtained after cross validation of training dataset
True  Positive :  212
True  Negative :  83
False Negative :  61
False Positive :  144
Accuracy for current value of K with maximum accuracy in cross validation is  59.0 %


<h2> Custom distance function - Using Jaccard Distance function</h2>

In [1176]:
#implementing jaccard
#version1
def jaccard(TrainDataSet, TestDataSet):
    InterSecSet = TrainDataSet.intersection(TestDataSet)
    return 1-float(len(InterSecSet)) / (len(TrainDataSet) + len(TestDataSet) - len(InterSecSet))

#version2
def DistJaccard(TrainDataSet, TestDataSet):
    return 1-float(len(TrainDataSet & TestDataSet)) / len(TrainDataSet | TestDataSet)

<h2> Function to calculate distance between any given test data with all the training samples based on the custom distance function to be used </h2>

In [1177]:
def getDistanceUsingCustomDistanceFunction(TestData, TrainingClassSet, TrainingDataSet, distanceFunctionToBeUsed='default'):
    distanceCalcList = defaultdict(list)
    
    index = 0
    
    for data in TrainingDataSet:
        
        if (distanceFunctionToBeUsed == 'jaccard'):
            distance = DistJaccard(set(data), set(TestData))
            distanceCalcList[distance].append(int(TrainingClassSet[index]))
        else:
            matchCount=0

            for word in TestData:
                if word in data:
                    matchCount+=1

            if (matchCount != 0): 
                distanceCalcList[1/matchCount].append(int(TrainingClassSet[index]))
            
        index+=1

    return distanceCalcList

In [1178]:
def performNearestNeighbourSearchForTestForCustomDistFunc(k, distanceFuncType='default'):
    print("using function type ", distanceFuncType)
    df = pd.DataFrame(columns=['Expected Class', 'Predicted Class'])
    correct = 0
    error = 0
    index1 = 0 #index to be used as reference for Expected class
    FP = 0 #False positive 
    FN = 0 #False Negative 
    TP = 0 #True positive
    TN = 0 #True Negative
    
    for testSample in TestWordsSet:
        ClassLabel = 0 #Label assigned to the current class

        DistanceCalc = getDistanceUsingCustomDistanceFunction(testSample, TrainingClassesSet, TrainingWordsSet, distanceFuncType)

        sortedKeys = sorted(DistanceCalc) 

        #Handling in case if k==1
        if (k == 1):
            if (len(DistanceCalc[sortedKeys[0]])) > 1:
                sumVal = sum(DistanceCalc[sortedKeys[0]])
                Mean= sumVal/(len(DistanceCalc[sortedKeys[0]]))

                if Mean >= 0.5:
                    ClassLabel = 1
                else:
                    ClassLabel = 0
            else:
                ClassLabel = DistanceCalc[sortedKeys[0]][0]
        else:
            
            #Handle condition for value of K more than 2
            NN = k
            TotalNN = 0
            sumVal = 0
            
            for key in sortedKeys:
                
                if (len(DistanceCalc[key])) > 1:
                    sumVal += sum(DistanceCalc[key])
                    TotalNN += len(DistanceCalc[key])
                else:
                    sumVal += DistanceCalc[key][0]
                    TotalNN += 1
                    
                NN = NN-len(DistanceCalc[key])
                
                if NN <= 0:
                    #print(NN)
                    break
            
            Mean = sumVal/TotalNN
            
            if Mean >= 0.5:
                ClassLabel = 1
            else:
                ClassLabel = 0
                    
        LabelList = [int(TestClassesSet[index1]), ClassLabel]
        
        tempVal = int(TestClassesSet[index1])
        if (tempVal == ClassLabel):
            correct+=1
        else:
            error+=1
        
        if (tempVal == 1):
            if (ClassLabel == 1):
                TP += 1
            else:
                FN += 1
        else:
            if (ClassLabel == 0):
                TN += 1
            else:
                FP+=1
                
        
        df.loc[len(df)] = LabelList
        
        index1 += 1 
    
    print(correct, " ", error)
    
    return df, TP, FN, TN, FP
    

<h2>Performing K-NN for test set using Custom distance function i.e. Jaccard distance function</h2>

<h3> For k=1 </h3>

In [1179]:
k=1
df, TP, FN, TN, FP = performNearestNeighbourSearchForTestForCustomDistFunc(k, 'jaccard')

print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

#Calculate the True Positive rate for k=1
print("True positive rate for k=1 is ", TP/(TP+FN))

#Calculate the False positive rate for k=1
print("False positive rate for k=1 is ", FP/(FP+TN))

#Calculate the accuracy for k=1
print("Accuracy for k=1 is ", ((TP+TN)/(TP+TN+FP+FN)) * 100, "%")

df.head(20)

using function type  jaccard
321   179
True  Positive :  206
True  Negative :  115
False Negative :  67
False Positive :  112
True positive rate for k=1 is  0.7545787545787546
False positive rate for k=1 is  0.4933920704845815
Accuracy for k=1 is  64.2 %


,Expected Class,Predicted Class
0,1,1
1,1,0
2,1,1
3,0,0
4,0,0
5,0,1
6,1,1
7,1,0
8,0,1
9,1,1


<h3> For k=5 </h3>

In [1180]:
k=5
df, TP, FN, TN, FP = performNearestNeighbourSearchForTestForCustomDistFunc(k, 'jaccard')

print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

#Calculate the True Positive rate for k=5
print("True positive rate for k=5 is ", TP/(TP+FN))

#Calculate the False positive rate for k=5
print("False positive rate for k=5 is ", FP/(FP+TN))

#Calculate the accuracy for k=5
print("Accuracy for k=5 is ", ((TP+TN)/(TP+TN+FP+FN)) * 100, "%")

df.head(20)

using function type  jaccard
334   166
True  Positive :  220
True  Negative :  114
False Negative :  53
False Positive :  113
True positive rate for k=5 is  0.8058608058608059
False positive rate for k=5 is  0.4977973568281938
Accuracy for k=5 is  66.8 %


,Expected Class,Predicted Class
0,1,1
1,1,1
2,1,1
3,0,0
4,0,1
5,0,1
6,1,1
7,1,1
8,0,1
9,1,1


<h2>Using Custom input for selecting any given value of K and selecting the distance function required (i.e Either the default function or using the Custom distance function i.e. Jaccard)</h2>

In [1155]:
k=1
funcType=1
func='jaccard'

k = int(input("Please input value for K: "))
funcType = int(input("Please select Func type (1 -> jaccard, 2 -> default): "))

if (funcType == 1):
    func = 'jaccard'
else:
    func = 'default'

df, TP, FN, TN, FP = performNearestNeighbourSearchForTestForCustomDistFunc(k, func)

print("True  Positive : ", TP)
print("True  Negative : ", TN)
print("False Negative : ", FN)
print("False Positive : ", FP)

df.head()

Please input value for K: 1
Please select Func type (1 -> jaccard, 2 -> default): 2
using function type  default
302   198
True  Positive :  209
True  Negative :  93
False Negative :  64
False Positive :  134


,Expected Class,Predicted Class
0,1,1
1,1,0
2,1,1
3,0,1
4,0,0
